In [1]:
import torch
import struct
import numpy as np
import matplotlib.pyplot as plt

In [2]:
def shannon_entropy(tensor: torch.Tensor, base: float = 2.0) -> float:
    abs_tensor = torch.abs(tensor)
    probs = abs_tensor / abs_tensor.sum()
    probs = probs[probs > 0]  # Игнорируем нулевые элементы
    if probs.numel() == 0:
        return 0.0
    if base == 2:
        log_fn = torch.log2
    elif base == torch.e:
        log_fn = torch.log
    else:
        log_fn = lambda x: torch.log(x) / torch.log(torch.tensor(base))
    entropy = -torch.sum(probs * log_fn(probs))
    return entropy.item()

In [67]:
def test(tensor):
    # Вычисляем энтропию в битах (по умолчанию)
    entropy_bits = shannon_entropy(tensor)
    print(f"Энтропия (биты): {entropy_bits:.4f}")
    # Вычисляем энтропию в натсах (натуральные единицы)
    entropy_nats = shannon_entropy(tensor, base=torch.e)
    print(f"Энтропия (наты): {entropy_nats:.4f}")

def bitflip_float(value, n: int, dtype: str = "float16"):
    import struct
    import numpy as np
    value = np.array(value)
    n = 15 - n
    if dtype == "float16":
        bits = np.frombuffer(value.tobytes(), dtype=np.uint16)[0]  # 16-битное представление
    elif dtype == "bfloat16":
        bits = np.frombuffer(np.array(value, dtype=np.float32).tobytes(), dtype=np.uint32)[0] >> 16  # Берем старшие 16 бит float32
    else:
        raise ValueError("dtype должен быть 'float16' или 'bfloat16'")
    if not (0 <= n < 16):
        raise ValueError("Позиция бита должна быть в диапазоне 0-15")
    flipped_bits = bits ^ (1 << n)
    if dtype == "float16":
        flipped_bytes = struct.pack("H", flipped_bits)
        return np.frombuffer(flipped_bytes, dtype=np.float16)[0]
    elif dtype == "bfloat16":
        flipped_value = (flipped_bits << 16)
        return np.frombuffer(struct.pack("I", flipped_value), dtype=np.float32)[0]

def make_fault(tensor, index, dtype):
    source_value = tensor[tuple(np.random.randint(tensor.shape))].item()
    fault_value = bitflip_float(source_value, index, dtype).item()
    print("[Fault injected]   source value: {}, fault value: {}".format(source_value, fault_value))
    tensor[tuple(np.random.randint(tensor.shape))] = fault_value
    return tensor

def make_fault_tensor(tensor1, tensor2, result, index, dtype, position):
    if position == 'tensor1':
        tensor1 = make_fault(tensor1, index, dtype)
    elif position == 'tensor2':
        tensor2 = make_fault(tensor2, index, dtype)
    elif position == 'result':
        result = make_fault(result, index, dtype)
    else:
        raise ValueError("Position should be: tensor1 or tensor2 or result")
    return tensor1, tensor2, result

In [85]:
m, n, k = 300, 500, 1000
type_ = torch.bfloat16
type_str = "bfloat16"

tensor1 = torch.randn(m, n).type(type_) * 1e-9
tensor2 = torch.randn(n, k).type(type_) * 1e-9
result = torch.matmul(tensor1, tensor2)


print("Tensor before fault")
test(result)


print("Tensor after fault")
fault_pattern = 'tensor1'
index = 1


tensor1, tensor2, result = make_fault_tensor(tensor1, tensor2, result, index, type_str, fault_pattern)
result = torch.matmul(tensor1, tensor2)
test(result)
# if fault_pattern == 'tensor1':
#     test(tensor1)
# elif fault_pattern == 'tensor2':
#     test(tensor2)
# else:
#     test(result)


Tensor before fault
Энтропия (биты): 17.7500
Энтропия (наты): 12.3125
Tensor after fault
[Fault injected]   source value: 1.2951204553246498e-09, fault value: 4.407066539855954e+29
Энтропия (биты): 9.5625
Энтропия (наты): 6.6250
